# HMetaD Group-Level Tutorial

This notebook demonstrates a basic hierarchical Bayesian HMeta-d fit on simulated data.
The example mirrors the structure of the RHMetaD regression tutorial, but uses the
standard group-level HMeta-d model.


In [ ]:
import numpy as np
import arviz as az
import metadpy
from scipy.stats import norm

from metadpy.bayesian import hmetad


In [ ]:
print('metadpy version:', metadpy.__version__)


## Simulate counts for multiple subjects

We simulate a small group (S=6) with K=4 confidence ratings. The simulation
generates type-2 multinomial counts (CR/FA/M/H) from a simple SDT model
with meta-d roughly aligned to d′ for clarity.


In [ ]:
def _clamp_probs(p, tol=1e-5):
    p = np.clip(p, tol, None)
    return p / p.sum()


def simulate_counts(rng, n_ratings, d1, meta_d, c1, c_s1, c_s2, n_cr, n_fa, n_m, n_h):
    n_subj = d1.shape[0]
    nR_S1 = np.zeros((n_subj, 2 * n_ratings), dtype=int)
    nR_S2 = np.zeros((n_subj, 2 * n_ratings), dtype=int)

    for idx in range(n_subj):
        s1_mu = -meta_d[idx] / 2.0
        s2_mu = meta_d[idx] / 2.0

        phi_c1_s1 = norm.cdf(c1 - s1_mu)
        phi_c1_s2 = norm.cdf(c1 - s2_mu)
        phi_cs1_s1 = norm.cdf(c_s1 - s1_mu)
        phi_cs1_s2 = norm.cdf(c_s1 - s2_mu)
        phi_cs2_s1 = norm.cdf(c_s2 - s1_mu)
        phi_cs2_s2 = norm.cdf(c_s2 - s2_mu)

        c_area_rs1 = phi_c1_s1
        i_area_rs2 = 1.0 - phi_c1_s1
        i_area_rs1 = phi_c1_s2
        c_area_rs2 = 1.0 - phi_c1_s2

        p_cr = np.concatenate((
            phi_cs1_s1[:1] / c_area_rs1,
            (phi_cs1_s1[1:] - phi_cs1_s1[:-1]) / c_area_rs1,
            np.array([(phi_c1_s1 - phi_cs1_s1[-1]) / c_area_rs1]),
        ))
        p_fa = np.concatenate((
            np.array([(phi_cs2_s1[0] - phi_c1_s1) / i_area_rs2]),
            (phi_cs2_s1[1:] - phi_cs2_s1[:-1]) / i_area_rs2,
            np.array([(1.0 - phi_cs2_s1[-1]) / i_area_rs2]),
        ))
        p_m = np.concatenate((
            phi_cs1_s2[:1] / i_area_rs1,
            (phi_cs1_s2[1:] - phi_cs1_s2[:-1]) / i_area_rs1,
            np.array([(phi_c1_s2 - phi_cs1_s2[-1]) / i_area_rs1]),
        ))
        p_h = np.concatenate((
            np.array([(phi_cs2_s2[0] - phi_c1_s2) / c_area_rs2]),
            (phi_cs2_s2[1:] - phi_cs2_s2[:-1]) / c_area_rs2,
            np.array([(1.0 - phi_cs2_s2[-1]) / c_area_rs2]),
        ))

        p_cr = _clamp_probs(p_cr)
        p_fa = _clamp_probs(p_fa)
        p_m = _clamp_probs(p_m)
        p_h = _clamp_probs(p_h)

        cr_counts = rng.multinomial(n_cr, p_cr)
        fa_counts = rng.multinomial(n_fa, p_fa)
        m_counts = rng.multinomial(n_m, p_m)
        h_counts = rng.multinomial(n_h, p_h)

        nR_S1[idx, :n_ratings] = cr_counts
        nR_S1[idx, n_ratings:] = fa_counts
        nR_S2[idx, :n_ratings] = m_counts
        nR_S2[idx, n_ratings:] = h_counts

    return nR_S1, nR_S2


In [ ]:
rng = np.random.default_rng(123)
n_subj = 6
n_ratings = 4

d1 = rng.normal(1.5, 0.2, size=n_subj)
meta_d = d1.copy()
c1 = 0.0
c_s1 = np.array([-1.5, -0.5, -0.1])
c_s2 = np.array([0.1, 0.5, 1.5])

nR_S1, nR_S2 = simulate_counts(
    rng=rng,
    n_ratings=n_ratings,
    d1=d1,
    meta_d=meta_d,
    c1=c1,
    c_s1=c_s1,
    c_s2=c_s2,
    n_cr=60,
    n_fa=40,
    n_m=40,
    n_h=60,
)


## Fit the group-level HMeta-d model

We pass subject-level count arrays directly. Setting a placeholder `subject`
name activates the group-level model when working from counts.


In [ ]:
model, idata = hmetad(
    nR_S1=nR_S1,
    nR_S2=nR_S2,
    nRatings=n_ratings,
    subject='subject',
    draws=200,
    tune=200,
    chains=1,
    random_seed=123,
    target_accept=0.9,
    progressbar=False,
    cores=1,
)
az.summary(idata, var_names=['mu_meta_d', 'sigma_meta_d'])


## Notes
- For trial-level data, pass a DataFrame plus the `subject` column name.
- Increase draws/chains for real analyses to improve posterior stability.
